# Part 2: Linear Classification (Theory)

Before we start to implement machine learning models in TensorFlow, we should first understand what their purpose is, and how they work. If you're already familiar with linear classification, feel free to jump directly into the other notebook.

## Why machine learning?

Computers typically follow a set of instructions that humans write for them - they do what they're told, and have no ability to learn. But sometimes, to achieve certain tasks, it becomes very difficult for humans to write these instructions precisely enough. For example, consider the case where you give the computer a picture and ask it, "Is this a cat or a dog?". It would be insane to write code that looks for certain pixels here, certain colors there, and eventually determine the answer, let alone account for dozens of different breeds of cats and dogs. But what if computers could solve this problem by themselves?

This is what machine learning is about. To phrase this problem more concretely, we are given a picture $x$ consisting of pixels. We want to the computer to find a function $f(x): \text{pictures} \to \text{labels}$, where $\text{labels} = \{\text{cat}, \text{dog}\}$. For all the diagrams in this notebook, shaded rectangles are things we want to find.

![](images/2a-1.png)

And to give the computer some help in finding $f(x)$, we'll specify what kind of structure we want $f(x)$ to have.

## Linear Classification

Just to have some numbers, we'll let all of our pictures be 2x2 grayscale pictures. Clearly this is unrealistic but the methods here can easily be generalized to larger pictures and those with color. That makes 4 pixels. Our input $x$ can thus be represented as a vector of length 4. Because we are classifying between cat and dog, we'll say that we have 2 output classes. $\hat{y} = f(x)$ is thus a vector of length 2. If the first entry in $\hat{y}$ is greater, we'll say the picture is a cat, and if the second entry is greater, we'll say it's a dog.

To go from a vector of length 4 to a vector of length 2, we can simply multiply by a $2 \times 4$ matrix $W$ (the weight matrix). Then we'll add an offset $b$ as well (the bias vector). Thus, we have:

$$Wx + b = \hat{y}$$

Visually, our model now looks like this:

![](images/2a-2.png)

Alternatively phrased, every class score in $\hat{y}$ is a linear combination of the pixels in $x$ with some offset:

$$W_{1, 1}x_1 + W_{1, 2}x_2 + W_{1, 3}x_3 + W_{1, 4}x_4 + b_1 = \hat{y}_1 = \hat{y}_{\text{cat}}\\
W_{2, 1}x_1 + W_{2, 2}x_2 + W_{2, 3}x_3 + W_{2, 4}x_4 + b_2 = \hat{y}_2 = \hat{y}_{\text{dog}}$$

What is this doing? Consider if $W_{1, 1}$ was positive. That means our cat score increases when there are high values in the pixel $x_1$. If $W_{1, 1}$ was negative, our cat score would decrease when there are high values in $x_1$. Our cat score looks at each pixel individually, and decides if each pixel is indicative of a cat as described. The dog score works exactly the same way.

With our model defined as such, the computer now needs to find the optimal weight matrix $W$ and the optimal bias vector $b$. As a point of notation, we'll use $f_{W, b}(x)$ to represent the function $f(x)$ given some $W$ and $b$.

> #### Exercise 1
> Say you have 28x28 grayscale images and you want to classify them into 10 classes. With linear classfication, what should be the dimensions of the matrix $W$ and the vector $b$?

## Loss Functions

How do we optimize a function in general? One way to find an optimal function $f_{W^*, b^*}(x)$ is to first define a loss function $L(f_{W, b}(x))$ that measures "how bad" $f_{W, b}(x)$ is. 0 loss is good, and really high loss is terrible. Once we've defined a suitable loss function, all that's left would be to find

$$\underset{W, b}{\operatorname{argmin}} L(f_{W, b}(x))$$

What would be a good metric for "how optimal" a function is? You might be tempted to use accuracy with preexisting data. That is, say we have hundreds or thousands of pictures of cats and dogs with their class already known. Then we run each picture through our model and compare how often the predicted class matches the true class of the picture. This is actually not a bad idea for our current task, but it turns out that minimizing the loss using accuracy as a metric is rather difficult. We'll come back to why this is later.

But the accuracy metric has one part right - we do need to use lots of preexisting data. There's no way around it. So we still want to somehow compare the true class vector $y$ and our predicted class vector $\hat{y}$. $y$ always has a structure called one-hot encoding. In one-hot encoding, the correct class has a value of 1 and all other classes have a value of 0. For example, with a picture of a cat, $y = \begin{bmatrix} 1 & 0 \end{bmatrix}^T$. But since $\hat{y}$ can have any values - large, small, positive, or negative - it's difficult to immediately compare $\hat{y}$ with $y$.

$y$ can be interpreted as a probability vector where the probablity of the correct class is 1, and the probability of all other classes is 0. We can normalize $\hat{y}$ so that it becomes a probability vector too, where all entries are positive and the sum equals 1. To make all entries positive, we'll first take $\operatorname{exp}(\hat{y}_i)$ for every $i$. Then we'll divide each entry by the sum of all entries, which makes the final sum 1. This is called the softmax function $\sigma$ and is thus written:

$$\sigma (\hat{y})_i = \frac{\operatorname{exp}(\hat{y}_i)}{\sum_k \operatorname{exp}(\hat{y}_k)}$$

> #### Exercise 2
> Compute the output of the softmax function if $\hat{y} = \begin{bmatrix} -0.01 & 0.01 \end{bmatrix}^{T}$ and if $\hat{y} = \begin{bmatrix} 5 & -4 \end{bmatrix}^{T}$. Interpret the results as probabilities in our model.

Note that $\operatorname{argmax} y$ is the position of the true class. So if we take $\sigma (\hat{y})_{\operatorname{argmax} y}$ we are looking at the predicted probability of the true class. If we predicted a probability of 1, there should be 0 loss, but if we predicted a low probability, the loss should be high. Taking the negative log achieves this effect.

$$L = -\operatorname{log}(\sigma (\hat{y})_{\operatorname{argmax}y})$$

We can verify that if the probability $\sigma (\hat{y})_{\operatorname{argmax}y}$ is 1, the loss is 0, and if the probability is low, like 0.05, the loss is much higher (about 3). This function is called cross-entropy loss. 

Here's how our model looks now:

![](images/2a-3.png)

Remember that our goal is still to classify new pictures $x$ without already having a $y$. Notice that our model still fits this goal. Even though we need $y$ to compute the loss, once we've minimized the loss for $W$ and $b$, we can throw out everything other than the $Wx + b$ and classify directly from $\hat{y}$.

Let's minimize the loss!

> #### Exercise 3
> With softmax and cross entropy loss, is it actually possible to have 0 loss? Why or why not? If we try to minimize loss as much as possible, what would $\hat{y}$ look like (in terms of the magnitude of numbers)?

## Gradient Descent

To gain some intution for our method to minimize the loss, imagine that the function we're minimizing is $f(x) = x^2 + 2x$.

![](images/2a-4.png)

First, we choose to start at any $x$ on this graph. Say we start at $0$. Then, following the slope of the line, we walk down and to the left until we hit $-1$, at which point we stop walking. If, say, we started at $-2$, we would walk down and to the right until we hit $-1$.

To formalize walking, we'll say that the following is taking one step:

$$ x := x - \epsilon f'(x) $$

$:=$ is the assignment operator, $f'(x)$ is the derivative of $f(x)$ at $x$, and $\epsilon$ is the "size" of our step. What are we doing here? First, $f'(x)$ tells us the slope of the line, or in other words, which way is up. Ignore $\epsilon$ for now. Because we want to go down, not up, we update $x$ by the negative of the slope. Ideally, we would travel by the negative of the derivative until we reach $x = -1$, where the derivative is $0$ and we stop moving.

Ignoring $\epsilon$ again, we test our method to see how it works. First, we start at $0$. The derivative of $f(x)$ is $2x + 2$, so at $0$, the $f'(0) = 2$. After one step, $x = 0 - 2 = -2$. $f'(-2) = -2$, so after our second step, $x = -2 - (-2) = 0$. We're back where we started!

![](images/2a-5.png)

That's why we introduce $\epsilon$, the step size (or learning rate as we'll soon see). If we take smaller steps, we'll be able to converge. Say we set $\epsilon = \frac{1}{4}$ and start again at $x = 0$. The first few steps would be:

$$ \begin{align}
x & := 0 - \frac{1}{4}f'(0) = 0 - \frac{1}{4}(2) = -\frac{1}{2} \\
x & := -\frac{1}{2} - \frac{1}{4}f'\left(\frac{1}{2}\right) = -\frac{1}{2} - \frac{1}{4}(1) = -\frac{3}{4} \\
x & := -\frac{3}{4} - \frac{1}{4}f'\left(\frac{3}{4}\right) = -\frac{3}{4} - \frac{1}{4}\left(\frac{1}{2}\right) = -\frac{7}{8}
\end{align}$$

We're converging to $-1$!

![](images/2a-6.png)

Let's recap what we just did here. First, we chose an initial value for $x$. The derivative tells you how to go up, but we want to go down. So we follow $x$ as it takes small steps in the opposite direction of the derivative. In the end, $x$ converges to the argmin of the function. This method is called gradient descent.

Just like in this toy example, we can also use gradient descent for our model. We don't get the pretty visualizations, but the concept is the same. First, initialize $W$ and $b$ to some numbers. Remember that our loss is a function of $f_{W, b}(x)$, so our loss is really also a just function of $W$, $b$, and $x$. Then, we can take the partial derivative of our loss $L$ with respect to $W$ and $b$. Take a small step in the opposite direction of the partial derivative for both $W$ and $b$. Repeat until convergence. 

"But wait!" you say. "Taking the derivative of this giant function by hand will be really hard! There's all sorts of crazy stuff here!" Luckily for us, TensorFlow will take the derivative of our model automatically. We'll leave the specifics of how that works to the next theory notebook, so for now, just know that we can optimize $f_{W, b}(x)$ by taking the partial derivative of the loss with respect to $W$ and $b$, then updating $W$ and $b$ accordingly.

Finally, we have a way to find $W$ and $b$. Our model is complete! Here's how it looks:

![](images/2a-7.png)